## Dependencies
* standard libraries: random, math, itertools
* numpy scipy
* keras / tensorflow

In [3]:
import random
import math
from itertools import permutations
import numpy as np
from scipy.spatial.transform import Rotation as R

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.backend()

Using TensorFlow backend.


'tensorflow'

## Model

In [5]:
all_rotation_orders = [''.join(axes) for axes in permutations({'x','y','z'})]
all_rotation_orders

['zxy', 'zyx', 'xzy', 'xyz', 'yzx', 'yxz']

In [6]:
def random_vector(n, mag):
    return 2.0 * mag * (0.5-np.random.random(size=n))
random_vector(10,5.0)

array([ 0.39354099, -2.51519837,  3.3430338 , -0.27819459, -1.9238247 ,
        3.78029716,  2.34341383, -4.85547446, -2.91900228,  1.04633776])

define a function to generate a single sample from the model.

sample is a tuple of:
* np.array with 9 matrix elements + 3 rotation angles
* string corresponding to chosen rotation order (element of all_rotation_orders)

In [7]:
def generate_sample_from_model():
    rotation_order = random.choice(all_rotation_orders)  # choose a rotation order
    angles = random_vector(3, 10.0)   # generate three random angles (in degrees)
    if (min(abs(angles)) < 0.1):     # reject any angles that are too small
        return generate_sample_from_model()
    matrix = R.from_euler(rotation_order, angles, degrees = True)  # generate matrix
    noise_angles = angles + random_vector(3, 0.1)    
    x = np.concatenate((matrix.as_dcm().flatten(), 
                        np.array(noise_angles)))
                        # np.array(np.cos(noise_angles)), 
                        # np.array(np.sin(noise_angles))))                         
    return (x, rotation_order)
generate_sample_from_model()

(array([ 9.85259481e-01,  1.69878163e-01,  2.01287259e-02, -1.69680941e-01,
         9.85436043e-01, -1.11437213e-02, -2.17286470e-02,  7.56399594e-03,
         9.99735291e-01,  1.18077686e+00, -9.68395470e+00,  3.48995296e-01]),
 'yzx')

define a function to generate multiple samples, suitable for training or testing.

for number N, sample is a tuple:
 * np.array Nx12 of x vectors (standardized zero mean)
 * np.array Nx6 one-hot vectors

In [8]:
def generate_samples(n):
    [xs, rotation_orders] = list(zip(*[generate_sample_from_model() for _ in range(n)]))
    xs = np.array(xs)
    xs -= xs.mean(0)    # standardize along sample dimension
    xs /= xs.std(0)
    ys = [all_rotation_orders.index(rotation_order) for rotation_order in rotation_orders]
    return xs, keras.utils.to_categorical(ys)
generate_samples(3)

(array([[ 0.69839686,  0.74051702,  0.58504971, -0.6734823 ,  1.34762182,
          0.59695347, -0.70265872, -0.47980557,  1.04083019, -1.35330207,
         -0.22532857,  0.10522089],
        [ 0.71578108,  0.67316178,  0.82250323, -0.74020648, -0.30241279,
         -1.4087627 , -0.71154554,  1.39200519, -1.34957394,  1.03222119,
         -1.09643467,  1.1687398 ],
        [-1.41417795, -1.4136788 , -1.40755294,  1.41368878, -1.04520903,
          0.81180923,  1.41420425, -0.91219963,  0.30874374,  0.32108087,
          1.32176324, -1.27396069]]), array([[0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]], dtype=float32))

generate training and testing data from the model

In [9]:
x_train, y_train = generate_samples(10000)
x_test, y_test = generate_samples(1000)
x_train.shape, y_train.shape

((10000, 12), (10000, 6))

now create an MLP as the model

In [10]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 4,166
Trainable params: 4,166
Non-trainable params: 0
_________________________________________________________________


set up the optimizer and compile the model

In [11]:
adam = Adam(lr=0.001, decay=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

now fit the data using the training samples

In [12]:
model.fit(x_train, y_train, epochs=200, batch_size=256)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
10000/10000 [==============================] - 0s 32us/step - loss: 1.7356 - acc: 0.2786
Epoch 2/200
10000/10000 [==============================] - 0s 6us/step - loss: 1.4911 - acc: 0.4860
Epoch 3/200
10000/10000 [==============================] - 0s 6us/step - loss: 1.0709 - acc: 0.6495
Epoch 4/200
10000/10000 [==============================] - 0s 8us/step - loss: 0.7180 - acc: 0.7702
Epoch 5/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.5320 - acc: 0.8333
Epoch 6/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.4335 - acc: 0.8659
Epoch 7/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.3677 - acc: 0.8920
Epoch 8/200
10000/10000 [==============================] - 0s 7us/step - loss: 0.3259 - acc: 0.9007
Epoch 9/200
10000/10000 [==============================] - 0s 7us/step - loss: 0.2895 - acc: 0.9153
Epoch 10/200
10000/10000 [=========================

10000/10000 [==============================] - 0s 9us/step - loss: 0.0499 - acc: 0.9832
Epoch 80/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0432 - acc: 0.9877
Epoch 81/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0436 - acc: 0.9872
Epoch 82/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0418 - acc: 0.9871
Epoch 83/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0385 - acc: 0.9898
Epoch 84/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0377 - acc: 0.9909
Epoch 85/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0410 - acc: 0.9885
Epoch 86/200
10000/10000 [==============================] - 0s 7us/step - loss: 0.0392 - acc: 0.9889
Epoch 87/200
10000/10000 [==============================] - 0s 7us/step - loss: 0.0371 - acc: 0.9909
Epoch 88/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0395 - acc: 0.9888
Epo

10000/10000 [==============================] - 0s 7us/step - loss: 0.0431 - acc: 0.9855
Epoch 160/200
10000/10000 [==============================] - 0s 7us/step - loss: 0.0242 - acc: 0.9911
Epoch 161/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0217 - acc: 0.9929
Epoch 162/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0216 - acc: 0.9933
Epoch 163/200
10000/10000 [==============================] - 0s 7us/step - loss: 0.0218 - acc: 0.9928
Epoch 164/200
10000/10000 [==============================] - 0s 7us/step - loss: 0.0227 - acc: 0.9933
Epoch 165/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0276 - acc: 0.9902
Epoch 166/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0206 - acc: 0.9940
Epoch 167/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0275 - acc: 0.9907
Epoch 168/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0300 - acc: 

In [13]:
score = model.evaluate(x_test, y_test, batch_size=256)
print(score)

1000/1000 [==============================] - 0s 49us/step
[0.16887659102678299, 0.941000006198883]


In [29]:
import json
import csv
with open('model_config.json', 'w') as modelfile:  
    json.dump(model.get_config(), modelfile)

In [28]:
def rows_of(array):
    if len(array.shape) > 1:
        for n in range(array.shape[0]):
            yield (n, array[n])
    else:
        yield (0, array)
        
with open('weights.csv', 'w', newline='') as csvfile:
    weightswriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for layer in model.layers:
        for weight_variable in layer.weights:
            weight_values = K.batch_get_value(weight_variable)
            for (row_num, row_array) in rows_of(weight_values):
                if len(weight_values.shape) > 1:
                    row_count = weight_values.shape[0]
                    col_count = weight_values.shape[1]
                else:
                    row_count = 1
                    col_count = weight_values.shape[0]                    
                csv_row = [layer.name, weight_variable.name,
                           row_count, col_count, row_num] + list(row_array)
                weightswriter.writerow(csv_row)